# Algoritmo de Maximización de la Esperanza

<img style="float: right; margin: 0px 0px 15px 15px;" src="https://upload.wikimedia.org/wikipedia/commons/3/3d/EM_Process.jpg" width="500px" height="300px" />


> Con los preliminares vistos en el anterior notebook, estamos listos para entrarle a la explicación del algoritmo de maximización de la esperanza.

> Este interesante algoritmo sirve para entrenar los parámetros de una gran mayoría de los modelos con variables latentes, incluido el modelo de mezclas Gaussianas.

> **Objetivos:**
> - Entender la forma general del algoritmo de maximización de la esperanza.
> - Obtener detalladamente la elección en el paso E.
> - Obtener detalladamente la elección en el paso M.

> **Referencias:**
> - Bayesian Methods for Machine Learning course, HSE University, Coursera.

## 1. Forma general del algoritmo de maximización de la experanza

Comenzamos por establecer la situación general sobre la que definiremos el algoritmo de maximización de la esperanza.

Tenemos un modelo de las variables $t_i$ y $x_i$, para $i=1,\dots,N$ ($N$ puntos de datos) de la siguiente manera:

![latent](figures/latent_model.png)

donde

- $t_i \in \{1, \dots, k\}$: es una variable latente que determina el valor de
- $x_i \in \mathbb{R}^{d}$: variable(s) medidas,
- $\theta$: son los parámetros del modelo.

Tenemos que la verosimilitud marginal es:

$$
p(x_i | \theta) = \sum_{c=1}^{k} p(x_i, t_i=c | \theta) = \sum_{c=1}^{k} p(x_i | t_i=c, \theta) p(t_i=c | \theta).
$$

Como habíamos visto en nuestro ejemplo de mezclas Gaussianas, podemos maximizar la función de verosimilitud marginal (marginal porque marginalizamos respecto a la variable latente) para encontrar los parámetros:

$$
\arg \max_{\theta} p(X | \theta).
$$

Recordamos el truco común, lo anterior es igual a:

$$
\arg \max_{\theta} \log p(X | \theta).
$$

Adicionalmente, suponiendo muestras i.i.d., tenemos que:

\begin{align}
\log p(X | \theta) &= \sum_{i=1}^{N} \log p(x_i | \theta) \\
                   &= \sum_{i=1}^{N} \log \left(\sum_{c=1}^{k} p(x_i, t_i=c | \theta)\right).
\end{align}

En general, vimos que es posible maximizar esta función usando un método numérico estándar (gradiente descendiente). Sin embargo, se introducen ciertas complejidades que queremos evitar.

**¿Qué hacemos entonces?** La idea general es la siguiente:

1. Encontramos una cota inferior para esta expresión usando la desigualdad de Jensen(los detalles vendrán después):

    \begin{align}
    \log p(X | \theta) &= \sum_{i=1}^{N} \log p(x_i | \theta) \\
                       &= \sum_{i=1}^{N} \log \left(\sum_{c=1}^{k} p(x_i, t_i=c | \theta)\right) \geq \mathcal{L}(\theta).
    \end{align}

   Obviamente, la función $\mathcal{L}(\theta)$ no será cualquier cota inferior, sino una que sea sencilla de maximizar. ¿Cómo nos aseguramos que $\mathcal{L}(\theta)$ sea una buena elección? (explicar problemática en el pizarrón).
   
2. En realidad, lo que hacemos es elegir no solo una cota, sino una familia de cotas inferiores que nos permitirá elegir **la mejor cota inferior**.

Para esto, introducimos una distribución artificial positiva $q(t) > 0$ (**distribución variacional**):

\begin{align}
\log p(X | \theta) &= \sum_{i=1}^{N} \log p(x_i | \theta) \\
                   &= \sum_{i=1}^{N} \log \left(\sum_{c=1}^{k} p(x_i, t_i=c | \theta)\right) \\
                   &= \sum_{i=1}^{N} \log \left(\sum_{c=1}^{k} \underbrace{\frac{q(t_i=c)}{q(t_i=c)}}_{1} p(x_i, t_i=c | \theta)\right). \\
\end{align}

Recordamos la desigualdad de Jensen: 

> *Proposición.* Sea $f:\Omega \subseteq \mathbb{R}^{k} \to \mathbb{R}$ una función cóncava. Entonces para cualquier selección de números $\alpha_i \geq 0$, con $i = 1, \dots, m$, tales que $\sum_{i=1}^m \alpha_i = 1$ y cualquier selección de elementos $x_i \in \Omega$, con $i = 1, \dots, m$, se tiene que:
> 
> $$
  f\left(\sum_{i=1}^{m} \alpha_i x_i\right) \geq \sum_{i=1}^m \alpha_i f(x_i)
  $$
  
Tomando en calidad de:
- la función cóncava $f(\cdot) = \log(\cdot)$,
- los escalares no negativos $\alpha_i = q(t_i=c)$, tales que $\sum_{i=1}^m q(t_i=c) = 1$ ($q$ es una distribución de probabilidad,
- los argumentos $x_i=\frac{1}{q(t_i=c)} p(x_i, t_i=c | \theta)$,

tenemos que:

\begin{align}
\log p(X | \theta) &= \sum_{i=1}^{N} \log p(x_i | \theta) \\
                   &= \sum_{i=1}^{N} \log \left(\sum_{c=1}^{k} p(x_i, t_i=c | \theta)\right) \\
                   &= \sum_{i=1}^{N} \log \left(\sum_{c=1}^{k} \underbrace{\frac{q(t_i=c)}{q(t_i=c)}}_{1} p(x_i, t_i=c | \theta)\right) \qquad \text{(desigualdad de Jensen)}\\
                   &\geq \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \log \left(\frac{1}{q(t_i=c)} p(x_i, t_i=c | \theta)\right) := \mathcal{L}(q, \theta).
\end{align}

Lo que hemos conseguido con este procedimiento es generar una familia de cotas inferiores $\mathcal{L}(q, \theta)$ (tenemos una cota inferior para cada distribución $q$), es decir:

$$
\log p(X | \theta) \geq \mathcal{L}(q, \theta) \quad \text{para toda }q
$$

De esta forma, podemos elegir la mejor distribución $q$.

**¿En qué sentido?**

Bueno, pues una propiedad deseable es que la cota inferior "no sea muy inferior" a lo que estamos intentando acotar (ilustrar en el pizarrón). De manera que, dado un valor previo de los parámetros $\theta_k$, podemos elegir la siguiente distribución $q_{k+1}$ haciendo:

$$
q^{j+1} = \arg \max_{q} \mathcal{L}(q, \theta^j) \Rightarrow \text{ E-step.}
$$

Una vez elegida la distribución artificial $q = q_{j+1}$, podemos elegir $\theta$ para maximizar la cota inferior:

$$
\theta^{j+1} = \arg \max_{\theta} \mathcal{L}(q^{j+1}, \theta) \Rightarrow \text{ M-step.}
$$

Es decir, estamos aplicando una estrategia del tipo **divide y conquistarás**.

Tenemos una función compleja (como la <font color=red>roja</font> en la gráfica de abajo), y lo que hacemos es encontrar una función fácil de maximizar que sea una cota inferior de la función compleja (como la <font color=blue>azul</font> en la gráfica de abajo):

![img](https://upload.wikimedia.org/wikipedia/commons/3/3d/EM_Process.jpg)

## 2. E-step

Ya que conocemos la idea general, concentrémosnos en nuestro primer problema.

Encontramos una familia de cotas inferiores (cota inferior variacional) de la log-verosimilitud marginal:

$$
\log p(X | \theta) \geq \mathcal{L}(q, \theta) \quad \text{para toda }q
$$

y queremos encontrar la distribución $q$ que maximice:

$$
q^{j+1} = \arg \max_{q} \mathcal{L}(q, \theta^j).
$$

¿Cómo se ve esto gráficamente? (ver en el pizarrón)

De manera que el GAP entre la log-verosimilitud y la cota variacional es:

\begin{align}
0 \leq GAP & = \log p(X | \theta) - \mathcal{L}(q, \theta) \\
           & = \sum_{i=1}^{N} \log p(x_i | \theta) - \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \log \left(\frac{p(x_i, t_i=c | \theta)}{q(t_i=c)}\right) \\
           & = \sum_{i=1}^{N} \left(\log p(x_i | \theta) \underbrace{\sum_{c=1}^{k} q(t_i=c)}_{1} - \sum_{c=1}^{k} q(t_i=c) \log \left(\frac{p(x_i, t_i=c | \theta)}{q(t_i=c)} \right)\right) \\
           & = \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \left(\log p(x_i | \theta) - \log \left(\frac{p(x_i, t_i=c | \theta)}{q(t_i=c)} \right)\right) \\
           & = \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \log\left(\frac{p(x_i | \theta) q(t_i=c)}{p(x_i, t_i=c | \theta)} \right)
\end{align}

**Pregunta:** ¿Podemos simplificar la expresión $\frac{p(x_i | \theta)}{p(x_i, t_i=c | \theta)}$?

De manera que

\begin{align}
0 \leq GAP & = \log p(X | \theta) - \mathcal{L}(q, \theta) \\
           & = \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \log\left(\frac{q(t_i=c)}{p(t_i=c | x_i, \theta)} \right)
\end{align}

Recordamos la definición de **la divergencia de Kullback-Leibler**

> *Definición.* Dadas dos distribuciones de probabilidad, la divergencia de Kullback-Leibler se define como:
>
> $$
  \mathcal{KL}(q || p) = \sum_{x} q(x) \log \frac{q(x)}{p(x)} = E_{q(x)}\left[\log \frac{q(x)}{p(x)}\right]
  $$
>
> si las variables son discretas.

**Pregunta:** ¿A qué es igual la expresión $\sum_{c=1}^{k} q(t_i=c) \log\left(\frac{q(t_i=c)}{p(t_i=c | x_i, \theta)} \right)$?

Así,

\begin{align}
0 \leq GAP & = \log p(X | \theta) - \mathcal{L}(q, \theta) \\
           & = \sum_{i=1}^{N} \mathcal{KL}(q(t_i) || p(t_i|x_i, \theta)).
\end{align}
           
           
Recordando que lo que queríamos era:

$$
q^{j+1} = \arg \max_{q} \mathcal{L}(q, \theta^j) = \arg \min_{q} GAP(q, \theta^j) = \arg \min_{q}\sum_{i=1}^{N} \mathcal{KL}(q(t_i) || p(t_i|x_i, \theta^j))
$$

Además, la divergencia tiene las siguientes propiedades:

1. $\mathcal{KL}(q || q) = 0$
2. $\mathcal{KL}(q || p) \geq 0$

Por tanto, para minimizar lo anterior, hacemos:

$$
q^{j+1}(t_i) = p(t_i|x_i, \theta^j)
$$

con lo cual obtenemos **el óptimo global**. Es decir, debemos elegir la distribución variacional como la distribución posterior $p(t_i|x_i, \theta^j)$.

**Pregunta:** ¿Qué implica que $GAP=0$?

## 3. M-step

Ahora, centramos nuestra atención en el segundo problema.

Entonces, podemos resumir nuestro camino hasta acá de la siguiente manera:

1. Encontramos la cota inferior variacional para la log-verosimilitud marginal:

   $$
   \log p(X | \theta) \geq \mathcal{L}(q, \theta) = \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \log \left(\frac{p(x_i, t_i=c | \theta)}{q(t_i=c)}\right).
   $$
   
2. Encontramos la mejor cota inferior respecto a la distribución artificial $q$, teniendo los parámetros fijos (E-step):
   
   $$
   q^{j+1}(t_i) = \arg \max_{q} \mathcal{L}(q, \theta^j) =  p(t_i|x_i, \theta^j).
   $$

3. Ahora, queremos encontrar el máximo de esta mejor cota inferior respecto a los parámetros $\theta$ (dejando fija la distribución $q^{j+1}$ que recién hallamos):

   $$
   \theta^{j+1} = \arg \max_{\theta} \mathcal{L}(q^{j+1}, \theta)
   $$

Trabajemos con la expresión de $\mathcal{L}$:

\begin{align}
\mathcal{L}(q, \theta) & = \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \log \left(\frac{p(x_i, t_i=c | \theta)}{q(t_i=c)}\right) \\
                       & = \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \log \left(p(x_i, t_i=c | \theta)\right) - \underbrace{\sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \log \left(q(t_i=c)\right)}_{\text{no depende de }\theta}
\end{align}

De manera que:

\begin{align}
\arg \max_{\theta} \mathcal{L}(q, \theta) & = \arg \max_{\theta} \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \log \left(p(x_i, t_i=c | \theta)\right) \\
                                          & = \arg \max_{\theta} \sum_{i=1}^{N} \mathbb{E}_{q(t_i)} \left[\log p(x_i, t_i| \theta)\right]
\end{align}

Comúnmente, tendremos que la función de verosimilitud $p(x_i, t_i| \theta)$ es cóncava respecto a los parámetros (por ejemplo, una Gaussiana), y tiene un único máximo global fácil de encontrar (analíticamente).

En resumen, **el algoritmo de maximización de la esperanza** se resume como sigue:

- Para $j=1, 2, \dots$, repetir hasta la convergencia:

  1. E-step:
     
     $$
     q^{j+1}(t_i) = p(t_i|x_i, \theta^j).
     $$
     
  2. M-step
      
      $$
      \theta^{j+1} = \arg \max_{\theta} \sum_{i=1}^{N} \mathbb{E}_{q^{j+1}(t_i)} \left[\log p(x_i, t_i| \theta)\right]
      $$

## 4. Propiedades de convergencia

Ya tenemos las expresiones para nuestro algoritmo. Ahora nace la pregunta, ¿Qué tan bueno es?

Supongamos que estamos en la iteración $j=old$ en la gráfica siguiente:

![img](https://upload.wikimedia.org/wikipedia/commons/3/3d/EM_Process.jpg)

Entonces, en el *E-step* encontramos la mejor cota variacional (<font color=blue>curva azul</font>) $\mathcal{L}(q^{new}, \theta)$, la cual, en el *M-step*, maximizamos respecto a los parámetros, obteniendo $\theta^{new}$. Entonces:

\begin{align}
\log p(X | \theta^{new}) & \geq \mathcal{L}(q^{new}, \theta^{new}) \quad (\text{porque } \mathcal{L} \text{ es una cota inferior}) \\
                         & \geq \mathcal{L}(q^{new}, \theta^{old}) \quad (\text{porque } \theta^{new} \text{ es el máximo de la cota variacional}) \\
                         & = \log p(X | \theta^{old}) \quad (\text{porque } GAP=0)
\end{align}

Lo que acabamos de demostrar, es que a cada iteración del **algoritmo de maximización de la esperanza** siempre obtenemos un mejores parámetros (por lo menos no peores) que los que teníamos.

Por tanto, la convergencia a un punto crítico está asegurada.

## 5. Resumiendo...

- El algoritmo de **maximización de la esperanza** es un método para entrenar modelos de variable latente:

  ![](figures/latent_model.png)

- Dividimos un problema complejo de optimización en dos problemas más simples:
  1. Encontramos la distribución variacional que resulte en la mejor cota inferior para nuestra log-verosimilitud.
     - E-step:
       
       $$
       q^{j+1}(t_i) = p(t_i|x_i, \theta^j)
       $$
  2. Encontramos los parámetros que maximicen la mejor cota variacional.
     - M-step:
       
       $$
       \theta^{j+1} = \arg \max_{\theta} \sum_{i=1}^{N} \mathbb{E}_{q^{j+1}(t_i)} \left[\log p(x_i, t_i| \theta)\right]
       $$
- Convergencia garantizada a un máximo local.

- Nos ayuda con restricciones complejas: $\Sigma_c \succ 0$

Adicionalmente, este algoritmo se puede extender:

 - Restricción de las posibles distribuciones variacionales $q$.
 - Muestreo en el M-Step.

### ¡Suficientes matemáticas por hoy! Apliquemos este algoritmo a un ejemplo.

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>